<a href="https://colab.research.google.com/github/margot-bonilla/violent-behaviour-recognition/blob/master/notebooks/TFM_2024_Iteration_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Violence Detection

# Set Up

In [141]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [142]:
import os

# Crate necessary folders for the execution
info_path = '/content/Info'
if not os.path.exists(info_path):
    os.makedirs(info_path)

logs_path = '/content/logs'
if not os.path.exists(logs_path):
    os.makedirs(logs_path)

models_path = '/content/models/checkpoints'
if not os.path.exists(models_path):
    os.makedirs(models_path)

## Constant information to set the data and engineering resources

In [143]:
# Path information to load videos and annotations
import os

ROOT_PATH = r"/content/drive/MyDrive/UOC/TFM_2"
ANNOTATIONS_PATH = os.path.join(ROOT_PATH, "Annotations")
VIDEOS_PATH = os.path.join(ROOT_PATH, "Videos")
INFO_PATH = "Info"
MODELS_PATH = 'models'

LOGS_PATH = "logs"

DATASET_TRAIN_PATH = os.path.join(INFO_PATH, "violence_detection_train.hdf5")
DATASET_VAL_PATH = os.path.join(INFO_PATH, "violence_detection_val.hdf5")
DATASET_TEST_PATH = os.path.join(INFO_PATH, "violence_detection_test.hdf5")

## Krakov
KRAKOV_VIDEOS_PATH = os.path.join(ROOT_PATH, "Videos")
INCLUDE_KRAKOV = True

## Hockey
HOCKEY_VIDEOS_PATH = os.path.join(ROOT_PATH, "Hockey", "data")
INCLUDE_HOCKEY = True

## Movies
MOVIES_VIDEOS_PATH = os.path.join(ROOT_PATH, "movies")
INCLUDE_MOVIES = True

## RWF 2000
RWF2000_VIDEOS_PATH = os.path.join(ROOT_PATH, "2 - data")
INCLUDE_RWF2000 = True

## RWF 2000
VIOLENT_VIDEOS_PATH = os.path.join(ROOT_PATH, "violentFlows", "ForTal")
INCLUDE_VIOLENT = True

# Set the cells/process you want to run
CREATE_DATASET = True
PLOT_FRAME_DISTRIBUTION = False
IS_TEST_RUN = False
TEST_SIZE = 10
INSTALL_STUFF = False
LOAD_MODEL = False

EPOCHS = 10 if IS_TEST_RUN else 70
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 3 # 1 for grayscale 3 for RBG
BATCH_SIZE = 5

MAX_NUM_FRAMES = 40
APPLY_DATA_AUGMENTATION = True

In [144]:
# global variables
# total_videos will contain the full path of each of the videos
total_normal_videos = list()
total_violent_videos = list()

In [145]:
# Dedup videos
all_video_names = set()

In [146]:
def include_videos(video_path, normal_prefix):
  for v in os.listdir(video_path):
    total_path = os.path.join(video_path, v)
    if v.startswith(normal_prefix) and v not in all_video_names:
      total_normal_videos.append(total_path)
      all_video_names.add(v)
    elif v not in all_video_names:
      total_violent_videos.append(total_path)
      all_video_names.add(v)

In [147]:
if INCLUDE_KRAKOV:
  include_videos(os.path.join(KRAKOV_VIDEOS_PATH), 'Normal')

In [148]:
if INCLUDE_HOCKEY:
  include_videos(os.path.join(HOCKEY_VIDEOS_PATH), 'no')

In [149]:
if INCLUDE_MOVIES:
  for f in os.listdir(MOVIES_VIDEOS_PATH):
    folder_path_normal = os.path.join(MOVIES_VIDEOS_PATH, f, "NonViolence")
    folder_path_violent = os.path.join(MOVIES_VIDEOS_PATH, f, "Violence")
    for normal in os.listdir(folder_path_normal):
      if normal not in all_video_names:
        total_normal_videos.append(os.path.join(folder_path_normal, normal))
        all_video_names.add(normal)
    for violent in os.listdir(folder_path_violent):
      if violent not in all_video_names:
        total_violent_videos.append(os.path.join(folder_path_violent, violent))
        all_video_names.add(violent)

In [150]:
if INCLUDE_RWF2000:
  for f in os.listdir(RWF2000_VIDEOS_PATH):
    folder_path_normal = os.path.join(RWF2000_VIDEOS_PATH, f, "NonFight")
    folder_path_violent = os.path.join(RWF2000_VIDEOS_PATH, f, "Fight")
    for normal in os.listdir(folder_path_normal):
      if normal not in all_video_names:
        total_normal_videos.append(os.path.join(folder_path_normal, normal))
        all_video_names.add(normal)
    for violent in os.listdir(folder_path_violent):
      if violent not in all_video_names:
        total_violent_videos.append(os.path.join(folder_path_violent, violent))
        all_video_names.add(violent)

In [151]:
if INCLUDE_VIOLENT:
  for v in os.listdir(VIOLENT_VIDEOS_PATH):
    if v not in all_video_names:
      total_violent_videos.append(os.path.join(VIOLENT_VIDEOS_PATH, v))
      all_video_names.add(v)

In [152]:
# Dedup videos
print("all video names: ", len(all_video_names))
print("all dedup video names: ", len(total_normal_videos) + len(total_violent_videos))

all video names:  6678
all dedup video names:  6678


In [153]:
print(f'Total of normal videos included: {len(total_normal_videos)}')
print(f'Total of violent videos included: {len(total_violent_videos)}')

Total of normal videos included: 4623
Total of violent videos included: 2055


## Libraries that you may need to run the notebook

In [154]:
# Execute this if you are not able to install tensorflow properly
if INSTALL_STUFF:
    !pip install tensorflow[and-cuda]
    !pip install pydot
    !pip install graphviz

## Plot distribution

In [155]:
import cv2
import os
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import progressbar

def create_frame_distribution(videos):
    number_of_frames = list()
    print(f"Total videos: {len(videos)}")
    bar = progressbar.ProgressBar(maxval=len(videos), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    processed = 0
    for video in videos:
        try:
            cap = cv2.VideoCapture(video)
            property_id = int(cv2.CAP_PROP_FRAME_COUNT)
            length = int(cv2.VideoCapture.get(cap, property_id))
            number_of_frames.append(length)
            cap.release()

            processed += 1
            bar.update(processed)

        except Exception as e:
            print(f"ERROR: {e.message}")
    bar.finish()

    return number_of_frames

def plot_frame_distribution(number_of_frames):
    frame_counts = np.array(number_of_frames)
    print("Number of unique frame counts:", len(frame_counts))
    print("Max frames in a video:", frame_counts.max())
    print("Min frames in a video:", frame_counts.min())
    print("Average frames per video:", frame_counts.mean())
    print("Median frames per video:", np.median(frame_counts))
    mode_value = st.mode(frame_counts).mode
    print("Mode frames per video:", mode_value)
    print("Videos over 2k frames:", np.sum(frame_counts > 2000))
    print("Videos over 5k frames:", np.sum(frame_counts > 5000))

    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(frame_counts, bins=100, color='skyblue', edgecolor='black')

    # Add vertical lines for mean, median, and mode
    plt.axvline(frame_counts.mean(), color='red', linestyle='dashed', linewidth=1, label='Mean')
    plt.axvline(np.median(frame_counts), color='yellow', linestyle='dashed', linewidth=1, label='Median')
    plt.axvline(mode_value, color='green', linestyle='dashed', linewidth=1, label='Mode')

    # Highlight majority population range (for example, mean ± one standard deviation)
    lower_bound = frame_counts.mean() - frame_counts.std()
    upper_bound = frame_counts.mean() + frame_counts.std()
    plt.axvspan(lower_bound, upper_bound, color='orange', alpha=0.2, label='Majority Range')

    plt.title('Distribution of Number of Frames per Video')
    plt.xlabel('Number of Frames')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.show()

In [156]:
if PLOT_FRAME_DISTRIBUTION:
  number_of_frames = create_frame_distribution(total_normal_videos + total_violent_videos)
  plot_frame_distribution(number_of_frames)

# Process video data

In [157]:
from enum import Enum

# class syntax
class Label(Enum):
    VIOLENT = "Violent"
    NORMAL = "Normal"

In [158]:
import cv2
import mediapipe as mp
import h5py
import numpy as np
from scipy.interpolate import interp1d

def duplicate_frames(frames, target_count):
  # Duplicate the last available frame to fill missing frames
  last_frame = frames[-1]
  while len(frames) < target_count:
      frames.append(last_frame.copy())
  return frames

def interpolate_frames(frames, keypoints, max_frames):
  current_length = len(frames)
  if current_length == max_frames:
    return frames, keypoints

  frame_indices = np.arange(current_length)
  target_indices = np.linspace(0, current_length - 1, max_frames)

  interpolated_frames = []
  # Assuming frames of shape (H, W, C)
  for i in range(frames[0].shape[2]):
    f = interp1d(frame_indices, [frame[:,:,i] for frame in frames], axis=0, kind='linear')
    interpolated_channel = f(target_indices)
    if i == 0:
      interpolated_frames = np.zeros((max_frames, interpolated_channel.shape[1], interpolated_channel.shape[2], frames[0].shape[2]))
    interpolated_frames[:,:,:,i] = interpolated_channel

  interpolated_keypoints = []
  # Assuming 33 keypoints
  for i in range(len(keypoints[0])):
    f = interp1d(frame_indices, [keypoint[i] for keypoint in keypoints], axis=0, kind='linear')
    interpolated_keypoint = f(target_indices)
    interpolated_keypoints.append(interpolated_keypoint)

  interpolated_keypoints = np.array(interpolated_keypoints).transpose((1, 0, 2))

  return interpolated_frames, interpolated_keypoints


def normalize(frame):
  # Normalize the frame
  frame = frame.astype(np.float32) / 255.0

  return frame

def resize(frame, target_size=(IMG_WIDTH, IMG_HEIGHT)):
  # Resize all frames to same dimension
  frame = cv2.resize(frame, dsize=target_size)

  return frame

def convert_grayscale(frame):
  # Convert to grayscale
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  return frame



In [159]:
if INSTALL_STUFF:
  !pip install mediapipe

In [160]:
import cv2
import mediapipe as mp

def extract_pose_keypoints(frame):
  """
  Will add only body and basic poses
  """
  mp_pose = mp.solutions.pose
  pose = mp_pose.Pose()

  frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  results = pose.process(frame_rgb)
  if results.pose_landmarks:
    person_landmarks = results.pose_landmarks
    # keypoints = [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    keypoints = np.zeros((13, 3), dtype=np.float32)

    # Extract body keypoints based on their index
    keypoints[0] = (person_landmarks.landmark[1].x, person_landmarks.landmark[1].y, person_landmarks.landmark[1].z)  # Left shoulder
    keypoints[1] = (person_landmarks.landmark[2].x, person_landmarks.landmark[2].y, person_landmarks.landmark[2].z)  # Right shoulder
    keypoints[2] = (person_landmarks.landmark[3].x, person_landmarks.landmark[3].y, person_landmarks.landmark[3].z)  # Left elbow
    keypoints[3] = (person_landmarks.landmark[4].x, person_landmarks.landmark[4].y, person_landmarks.landmark[4].z)  # Right elbow
    keypoints[4] = (person_landmarks.landmark[5].x, person_landmarks.landmark[5].y, person_landmarks.landmark[5].z)  # Left wrist
    keypoints[5] = (person_landmarks.landmark[6].x, person_landmarks.landmark[6].y, person_landmarks.landmark[6].z)  # Right wrist
    keypoints[6] = (person_landmarks.landmark[8].x, person_landmarks.landmark[8].y, person_landmarks.landmark[8].z)  # left hip
    keypoints[7] = (person_landmarks.landmark[9].x, person_landmarks.landmark[9].y, person_landmarks.landmark[9].z)  # right hip
    keypoints[8] = (person_landmarks.landmark[10].x, person_landmarks.landmark[10].y, person_landmarks.landmark[10].z)  # Left knee
    keypoints[9] = (person_landmarks.landmark[11].x, person_landmarks.landmark[11].y, person_landmarks.landmark[11].z)  # Right Knee
    keypoints[10] = (person_landmarks.landmark[12].x, person_landmarks.landmark[12].y, person_landmarks.landmark[12].z)  # Left Angle
    keypoints[11] = (person_landmarks.landmark[13].x, person_landmarks.landmark[13].y, person_landmarks.landmark[13].z)  # Right Angle
    keypoints[12] = (person_landmarks.landmark[15].x, person_landmarks.landmark[15].y, person_landmarks.landmark[15].z)  # Neck
  else:
    # Assuming 33 keypoints for consistency
    keypoints = [(0, 0, 0)] * 13

  pose.close()

  return keypoints

In [161]:
def save_to_hdf5(hdf5_file, video_name, frames, keypoints, label):
  with h5py.File(hdf5_file, 'a') as f:
    # Check if the group for the video already exists
    if video_name in f:
      # Delete existing datasets within the group (if any)
      del f[f'{video_name}']
    # Create a new group for the video data
    group = f.create_group(video_name)

    # Save frames, keypoints, and label as datasets within the group
    group.create_dataset('frames', data=np.array(frames))
    group.create_dataset('keypoints', data=np.array(keypoints))
    group.create_dataset('label', data=np.array([label]), dtype='i')


In [162]:
# Function to process a single video and its annotation
import os
import cv2

def process_video(video_path, hdf5_file, video_name, label, max_frames):
  cap = cv2.VideoCapture(video_path)
  frames = []
  keypoints_list = []

  while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      break

    frame_resized = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
    frame_normalized = frame_resized / 255.0
    frames.append(frame_normalized)

    keypoints = extract_pose_keypoints(frame)
    keypoints_list.append(keypoints)

  cap.release()

  # Ensure consistency frame length (padding or truncating)
  num_frames = len(frames)
  if num_frames < max_frames:
    frames, keypoints_list = interpolate_frames(frames, keypoints_list, max_frames)
  elif num_frames > max_frames:
    segment_count = num_frames // max_frames
    for i in range(segment_count):
      segment_frames = frames[i * max_frames:(i + 1) * max_frames]
      segment_keypoints = keypoints_list[i * max_frames:(i + 1) * max_frames]
      save_to_hdf5(hdf5_file, f'{video_name}_segment_{i}', segment_frames, segment_keypoints, label)

    # Handle remaining frames if any
    if num_frames % max_frames != 0:
      remaining_frames = frames[segment_count * max_frames:]
      remaining_keypoints = keypoints_list[segment_count * max_frames:]
      frames, keypoints_list = interpolate_frames(remaining_frames, remaining_keypoints, max_frames)
      save_to_hdf5(hdf5_file, f'{video_name}_segment_{segment_count}', frames, keypoints_list, label)

    return
  save_to_hdf5(hdf5_file, video_name, frames, keypoints_list, label)

In [163]:
# Test save video with frames, keypoints, and label
video_path = '/content/drive/MyDrive/UOC/TFM_2/Hockey/data/fi107_xvid.avi'
hdf5_file = 'test.hdf5'
video_name = 'fi107_xvid'
label = 1
max_frames = 20
process_video(video_path, hdf5_file, video_name, label, max_frames)

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.10/dist-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


In [164]:
import h5py

def explore_hdf5_file(hdf5_file_path):
    def print_attrs(name, obj):
        print(f"{name}: {obj}")
        if isinstance(obj, h5py.Dataset):
            print(f"  Shape: {obj.shape}")
            print(f"  Data type: {obj.dtype}")
        elif isinstance(obj, h5py.Group):
            print(f"  Contains: {list(obj.keys())}")

    with h5py.File(hdf5_file_path, 'r') as f:
        f.visititems(print_attrs)

# Example usage
print("Test file contents:")
explore_hdf5_file(hdf5_file)



Test file contents:
fi107_xvid_segment_0: <HDF5 group "/fi107_xvid_segment_0" (3 members)>
  Contains: ['frames', 'keypoints', 'label']
fi107_xvid_segment_0/frames: <HDF5 dataset "frames": shape (20, 128, 128, 3), type "<f8">
  Shape: (20, 128, 128, 3)
  Data type: float64
fi107_xvid_segment_0/keypoints: <HDF5 dataset "keypoints": shape (20, 13, 3), type "<f8">
  Shape: (20, 13, 3)
  Data type: float64
fi107_xvid_segment_0/label: <HDF5 dataset "label": shape (1,), type "<i4">
  Shape: (1,)
  Data type: int32
fi107_xvid_segment_1: <HDF5 group "/fi107_xvid_segment_1" (3 members)>
  Contains: ['frames', 'keypoints', 'label']
fi107_xvid_segment_1/frames: <HDF5 dataset "frames": shape (20, 128, 128, 3), type "<f8">
  Shape: (20, 128, 128, 3)
  Data type: float64
fi107_xvid_segment_1/keypoints: <HDF5 dataset "keypoints": shape (20, 13, 3), type "<f8">
  Shape: (20, 13, 3)
  Data type: float64
fi107_xvid_segment_1/label: <HDF5 dataset "label": shape (1,), type "<i4">
  Shape: (1,)
  Data type

In [165]:
import progressbar


def process_and_save_videos(videos, train_file, val_file):
  """
  viceos: Tuple: (video_path: string, label: Label)
  train_file: hdf5 file for train dataset
  val_file: hdf5 file for val dataset
  """
  bar = progressbar.ProgressBar(maxval=len(videos), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
  bar.start()
  for i, (video_path, label) in enumerate(videos):
    video_name = video_path.split('/')[-1].split('.')[0]
    if i / len(videos) < 0.8:
      process_video(video_path, train_file, video_name, label, MAX_NUM_FRAMES)
    else:
      process_video(video_path, val_file, video_name, label, MAX_NUM_FRAMES)

    bar.update(i)
  bar.finish()

In [166]:
import random


def save_dataset(all_videos):
    try:
        # load less volume for testing purposes
        if IS_TEST_RUN:
            all_videos = random.sample(all_videos, TEST_SIZE)

        # Process normal videos
        process_and_save_videos(all_videos, DATASET_TRAIN_PATH, DATASET_VAL_PATH)

        total = len(all_videos)
        print(f'\nTotal videos processed: {total}, Train: {total * 0.8}, Val: {total * 0.2}')

    except Exception as e:
        print(f'ERROR: {e}')
        print('Datasets not saved properly')
        raise Exception(e)


    print("Datasets are saved in HDF5 files correctly")
    print(f"Train dataset: {DATASET_TRAIN_PATH}")
    print(f"Val dataset: {DATASET_VAL_PATH}")

In [ ]:
import random


# label all normal and violent videos and shuffle them
all_videos = list()
for normal in total_normal_videos:
  all_videos.append((normal, 0))
for violent in total_violent_videos:
  all_videos.append((violent, 1))

random.shuffle(all_videos)

save_dataset(all_videos)


[================                                                        ]  22%

In [ ]:
# Function to explore the datasets
def traverse_datasets(hdf_file):
    """Traverse all datasets across all groups in HDF5 file."""

    import h5py

    def h5py_dataset_iterator(g, prefix=''):
        for key in g.keys():
            item = g[key]
            path = '{}/{}'.format(prefix, key)
            if isinstance(item, h5py.Dataset):  # test for dataset
                yield (path, item)
            elif isinstance(item, h5py.Group):  # test for group (go down)
                yield from h5py_dataset_iterator(item, path)

    with h5py.File(hdf_file, 'r') as f:
        print(f'Number of keys {len(f.keys())}\n')
        for (path, dset) in h5py_dataset_iterator(f):
            print(path, dset)

    return None

# Model Development

## Generators

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_keypoints(frame, keypoints):
    plt.imshow(frame)
    for keypoint in keypoints:
        x, y, z = keypoint
        plt.scatter(x * frame.shape[1], y * frame.shape[0], c='r', s=10)
    plt.title('Frame with Keypoints')
    plt.show()

def check_generator_output(generator, batch_index=0):
    # Get the batch from the generator
    (frames_batch, keypoints_batch), labels_batch = generator[batch_index]

    print(f"Batch shape (frames): {frames_batch.shape}")
    print(f"Batch shape (keypoints): {keypoints_batch.shape}")
    print(f"Batch shape (labels): {labels_batch.shape}")

    # Print out the first frame and corresponding keypoints from the first sample in the batch
    first_frame = frames_batch[0][0]
    first_keypoints = keypoints_batch[0][0]

    # Plot the first frame
    plt.imshow(first_frame)
    for x, y, _ in first_keypoints:
        plt.scatter(x, y, c='red')
    plt.show()

    # Plot keypoints on the first frame of the first sample
    plot_keypoints(first_frame, first_keypoints)


In [ ]:
import matplotlib.pyplot as plt

def visualize_frames(generator, num_batches=1, num_frames=5):
    for i in range(num_batches):
        X, y = generator[i]
        for j in range(num_frames):
            plt.imshow(X[j][0, :, :, 0])
            plt.title(f"Label: {y[j]}")
            plt.show()


In [ ]:
def check_class_distribution(generator):
    all_labels = []
    for i in range(len(generator)):
        _, y = generator[i]
        all_labels.extend(y)

    unique, counts = np.unique(all_labels, return_counts=True)
    label_distribution = dict(zip(unique, counts))

    print("Class Distribution:")
    for label, count in label_distribution.items():
        print(f" - Label {label}: {count} samples")


In [ ]:
import h5py
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

class HDF5DataGenerator(Sequence):
    """
    Custom data generator for HDF5 data with video fragments and labels per video.
    """

    def __init__(self, hdf5_file, batch_size, target_size, max_frames, augmentation=None):
        self.hdf5_file = hdf5_file
        self.batch_size = batch_size
        self.target_size = target_size
        self.max_frames = max_frames
        self.augmentation = augmentation
        self.indices = self._get_indices()
        self.on_epoch_end()

    def _get_indices(self):
        # Load data information from the HDF5 file
        with h5py.File(self.hdf5_file, "r") as f:
            return list(f.keys())

    def __len__(self):
        # Return the number of batches per epoch
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        # Generate a single batch of data and labels
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Load data and labels for the current batch of videos
        X, X_keypoints, y = self.__data_generation(batch_indices)

        return [X, X_keypoints], y

    def on_epoch_end(self):
        # Shuffle indices for each epoch
        np.random.shuffle(self.indices)

    def __data_generation(self, batch_indices):
        X = np.zeros((self.batch_size, self.max_frames, *self.target_size, 3), dtype=np.float32)
        X_keypoints = np.zeros((self.batch_size, self.max_frames, 13, 3), dtype=np.float32)
        y = np.zeros((self.batch_size,), dtype=np.float32)

        with h5py.File(self.hdf5_file, 'r') as f:
            for i, idx in enumerate(batch_indices):
                frames = f[f'{idx}/frames'][:]
                keypoints = f[f'{idx}/keypoints'][:]
                label = f[f'{idx}/label'][()]

                # Pad or truncate frames to max_frames
                if frames.shape[0] < self.max_frames:
                    pad_width = self.max_frames - frames.shape[0]
                    frames = np.pad(frames, ((0, pad_width), (0, 0), (0, 0), (0, 0)), mode='constant')
                    keypoints = np.pad(keypoints, ((0, pad_width), (0, 0), (0, 0)), mode='constant')
                elif frames.shape[0] > self.max_frames:
                    frames = frames[:self.max_frames]
                    keypoints = keypoints[:self.max_frames]

                if self.augmentation:
                    augmented_frames = []
                    for frame in frames:
                        augmented_frame = self.augmentation.random_transform(frame)
                        augmented_frames.append(augmented_frame)
                    frames = np.array(augmented_frames)

                X[i] = frames
                X_keypoints[i] = keypoints
                y[i] = label

        return X, X_keypoints, y

# Example usage
max_frames = 50  # Example value, adjust as needed
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 128, 128, 3  # Example values, adjust as needed

# Create data generators
train_generator = HDF5DataGenerator(DATASET_TRAIN_PATH, batch_size=8, target_size=(IMG_HEIGHT, IMG_WIDTH), max_frames=max_frames)
val_generator = HDF5DataGenerator(DATASET_VAL_PATH, batch_size=8, target_size=(IMG_HEIGHT, IMG_WIDTH), max_frames=max_frames)

# Example function to check the generator output
def check_generator_output(generator, batch_index=0):
    (frames_batch, keypoints_batch), labels_batch = generator[batch_index]

    print(f"Batch shape (frames): {frames_batch.shape}")
    print(f"Batch shape (keypoints): {keypoints_batch.shape}")
    print(f"Batch shape (labels): {labels_batch.shape}")

    # Optionally visualize the first frame and keypoints
    first_frame = frames_batch[0][0]
    first_keypoints = keypoints_batch[0][0]

    plt.imshow(first_frame.astype(np.uint8))
    for x, y, _ in first_keypoints:
        plt.scatter(x, y, c='red')
    plt.show()

# Check the generator output
check_generator_output(train_generator, batch_index=0)


## Load the data

In [ ]:
def load_dataset_data(dataset_name):
    X = []
    y = []
    frame_lengths = []
    with h5py.File(dataset_name, "r") as hdf5_file:
        video_names = list(hdf5_file.keys())
        for video_name in video_names:
            # Access frames and labels datasets for the specific video
            frames = hdf5_file[f"{video_name}/frames"][:]
            labels = hdf5_file[f"{video_name}/labels"][:]

            # Append data and label to batch lists
            X.append(frames)
            y.append(labels)
            frame_lengths.append(frames.shape[0])

    return np.array(X), np.array(y), np.array(frame_lengths)

In [ ]:
def get_dataset_size(dataset_name):
  size = 0
  with h5py.File(dataset_name, 'r') as f:
    video_names = list(f.keys())
    for v in video_names:
      size += len(f[f'{v}/frames'])

  return size

get_dataset_size(DATASET_TRAIN_PATH)

In [ ]:
if INSTALL_STUFF:
    !pip install tensorboard

# launch this command in your terminal if you want to see the tensorboard
# !tensorboard --logdir=C:\Users\margo\OneDrive\UOC\projects\thesis\logs

# Open http://localhost:6006 in your browser to access tensorboard

## Training

### Training using generators

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

import os

def train_model_using_generators(model, epochs=EPOCHS):
    """Trains a model on the provided data using data generators.

    Args:
      model: The Keras model to be trained.
      epochs: Number of training epochs.

    Returns:
      The trained model, train history, and val generator
    """

    # Generators
    train_generator = HDF5DataGenerator(DATASET_TRAIN_PATH, BATCH_SIZE, target_size=(IMG_HEIGHT, IMG_WIDTH), max_frames=MAX_NUM_FRAMES)
    val_generator = HDF5DataGenerator(DATASET_VAL_PATH, BATCH_SIZE, target_size=(IMG_HEIGHT, IMG_WIDTH), max_frames=MAX_NUM_FRAMES)

    # Callbacks
    my_callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ModelCheckpoint(filepath=os.path.join(MODELS_PATH, 'checkpoints', 'model.{epoch:02d}-{val_loss:.2f}.h5'),
                        save_best_only=True),
        TensorBoard(log_dir=LOGS_PATH),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    ]

    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    train_history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=my_callbacks
    )

    return model, train_history, val_generator


## Model

In [ ]:
from keras.layers import Conv3D, Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dropout, Dense, BatchNormalization, MaxPooling3D
from keras.models import Sequential
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import os

def create_video_model_deprecated(input_shape=(MAX_NUM_FRAMES, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), num_classes=1):
    model = Sequential([
        Conv3D(32, (3, 3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling3D((1, 2, 2)),  # Pool only in spatial dimensions

        Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling3D((1, 2, 2)),  # Pool only in spatial dimensions

        Conv3D(128, (3, 3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling3D((1, 2, 2)),  # Pool only in spatial dimensions

        Conv3D(128, (3, 3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling3D((1, 2, 2)),  # Pool only in spatial dimensions

        TimeDistributed(Flatten()),  # Flatten the output of each time step
        LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01)),  # L2 regularization
        Dropout(0.5),  # Dropout to prevent overfitting

        LSTM(32, kernel_regularizer=l2(0.01)),  # Another LSTM layer for added depth
        Dropout(0.5),

        Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')  # Sigmoid for binary classification
    ])

    return model

# Create the video model
video_model_deprecated = create_video_model_deprecated()


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, MaxPooling3D, TimeDistributed, Flatten, LSTM, Dropout, Dense, concatenate
from tensorflow.keras.regularizers import l2

def create_video_model(input_shape_frames=(MAX_NUM_FRAMES, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                       input_shape_keypoints=(MAX_NUM_FRAMES, 13, 3), num_classes=1):

    # Frame input branch
    frames_input = Input(shape=input_shape_frames, name='frames_input')
    x = Conv3D(32, (3,3,3), activation='relu', padding='same')(frames_input)
    x = BatchNormalization()(x)
    x = MaxPooling3D((1,2,2))(x)
    x = Conv3D(64, (3,3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D((1,2,2))(x)
    x = Conv3D(128, (3,3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D((1,2,2))(x)
    x = Conv3D(128, (3,3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D((1,2,2))(x)
    x = TimeDistributed(Flatten())(x)
    x = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    x = LSTM(32, kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)

    # Keypoints input branch
    keypoints_input = Input(shape=input_shape_keypoints, name='keypoints_input')
    y = TimeDistributed(Flatten())(keypoints_input)
    y = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(y)
    y = Dropout(0.5)(y)
    y = LSTM(32, kernel_regularizer=l2(0.01))(y)
    y = Dropout(0.5)(y)
    y = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(y)
    y = Dropout(0.5)(y)

    # Combine both branches
    combined = concatenate([x, y])
    z = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(combined)
    z = Dropout(0.5)(z)
    z = Dense(num_classes, activation='sigmoid')(z)

    model = Model(inputs=[frames_input, keypoints_input], outputs=z)
    return model

# Create the model
input_shape_frames = (MAX_NUM_FRAMES, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
input_shape_keypoints = (MAX_NUM_FRAMES, 13, 3)
video_model = create_video_model(input_shape_frames=input_shape_frames, input_shape_keypoints=input_shape_keypoints, num_classes=1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, TimeDistributed, LSTM
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from scipy import stats as st
import progressbar

m = create_video_model()
m.summary()

def plot_model_diagram(model, file_path='model_diagram.png'):
    plot_model(model, to_file=file_path, show_shapes=True, show_layer_names=True)
    img = plt.imread(file_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

plot_model_diagram(m)

# Evaluation

## Offline Evaluation

In [ ]:
from keras.models import load_model

# Train the model
model = create_video_model()
if LOAD_MODEL:
    trained_model, train_history, val_generator = load_model(MODELS_PATH, 'basic_model.keras')
else:
    trained_model, train_history, val_generator = train_model_using_generators(model, epochs=EPOCHS)

model.save(os.path.join(MODELS_PATH, 'basic_model.keras'))

In [ ]:
import pickle

with open('train_history.pkl', 'wb') as f:
    pickle.dump(train_history.history, f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve, average_precision_score


def show_model_metrics(train_history, model, val_generator):
    # Access training and validation loss/accuracy
    train_loss = train_history.history["loss"]
    val_loss = train_history.history["val_loss"]
    train_acc = train_history.history["accuracy"]
    val_acc = train_history.history["val_accuracy"]

    # Plot loss
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label="Training Loss")
    plt.plot(val_loss, label="Validation Loss")
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label="Training Accuracy")
    plt.plot(val_acc, label="Validation Accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

    # Evaluate model on validation data
    y_true = []
    y_pred = []

    for batch_x, batch_y in val_generator:
        y_true.extend(batch_y)
        y_pred_batch = model.predict(batch_x)
        y_pred.extend(np.round(y_pred_batch).astype(int))

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Ensure both y_true and y_pred are binary and have the same shape
    if y_true.ndim > 1:
        y_true = y_true.flatten()
    if y_pred.ndim > 1:
        y_pred = y_pred.flatten()

    # Trim y_pred to match the length of y_true if needed
    if len(y_pred) > len(y_true):
        y_pred = y_pred[:len(y_true)]
    elif len(y_true) > len(y_pred):
        y_true = y_true[:len(y_pred)]

    # Classification report
    print("Classification Report:")
    print('y_true', y_true);
    print('y_pred', y_pred);
    print(classification_report(y_true, y_pred, target_names=['Normal', 'Violent']))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Violent'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()


    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    average_precision = average_precision_score(y_true, y_pred)

    # Plot Precision-Recall curve
    plt.figure()
    plt.step(recall, precision, where='post', label='Precision-Recall curve (AP = %0.2f)' % average_precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve')
    plt.legend(loc="lower left")
    plt.show()


show_model_metrics(train_history, trained_model, val_generator)